テキストを.npファイルに変換

In [ ]:
a=seq

In [ ]:
with open('RSD.txt', mode='a') as f:
    f.write('\n'+name+'\t'+str(len(a)))

In [ ]:
b=seq

In [ ]:
A=np.zeros(len(a)).astype('str')
for i in range(len(a)):
    A[i]=a[i]

B=np.zeros(len(b)).astype('str')
for i in range(len(b)):
    B[i]=b[i]

In [ ]:
X=A
Y=B

In [ ]:
#NF2の決定
E1=0
for i in range(10+1, 1000):
    E1 = E1 + (9/16)*( (1/4)**i )*(i**2)

NF2List=[1]
for NF2 in NF2List:
    E2=0
    for i in range(NF2+1, 1000):
        E2 = E2 + (9/16)*( (1/4)**i )*(i**2)
    if(len(seq)*E2 <= 1988*E1):
        break
    else:
        NF2List.append(NF2+1)
NF2=NF2List[-1]

In [ ]:
#パラメータ決定
lengthOfRepeat=2000
NF1=0
#NF2=10
L1 = np.round(lengthOfRepeat/20).astype(np.int)
density = (2*NF2+1+1+1)/L1
L1_R = np.round(lengthOfRepeat/4).astype(np.int)
L2 = np.round(lengthOfRepeat/4).astype(np.int)
minLength = np.round(lengthOfRepeat/2).astype(np.int)
Ratio=1.5
N_fourier=10

Yに'N'を付け加えてXの長さにそろえる

In [ ]:
Z=np.repeat('N',len(X)-len(Y))
Y_=np.concatenate([Y, Z])
Y=Y_

ATGCの4種類の行を作る

In [ ]:
A_Line=np.zeros(len(Y), dtype=np.int8)
for i in range(len(Y)):
    if(Y[-i]=='A'):
        A_Line[i]=1

T_Line=np.zeros(len(Y), dtype=np.int8)
for i in range(len(Y)):
    if(Y[-i]=='T'):
        T_Line[i]=1

G_Line=np.zeros(len(Y), dtype=np.int8)
for i in range(len(Y)):
    if(Y[-i]=='G'):
        G_Line[i]=1

C_Line=np.zeros(len(Y), dtype=np.int8)
for i in range(len(Y)):
    if(Y[-i]=='C'):
        C_Line[i]=1

行列を作る

In [ ]:
Matrix=np.zeros([len(X),len(Y)], dtype=np.int8)
for i in range(len(X)):
    if(X[i]=='A'):
        Matrix[i]=np.roll(A_Line,i)
    elif(X[i]=='T'):
        Matrix[i]=np.roll(T_Line,i)
    elif(X[i]=='G'):
        Matrix[i]=np.roll(G_Line,i)
    elif(X[i]=='C'):
        Matrix[i]=np.roll(C_Line,i)

n連続の1をnに置換

In [ ]:
Hight=Matrix.shape[0]
Width=Matrix.shape[1]
Matrix[:,0]=np.zeros(Hight, dtype=np.int8)

In [ ]:
Matrix=Matrix.T
Matrix=Matrix.astype('int16')

In [ ]:
@jit(i2[:,:](i2[:,:]), nopython=True)
def f(M):
    for i in range(M.shape[0]):
        oneCount=0
        posi=0
        for j in range(M.shape[1]):
            if(M[i,j]==1):
                if(oneCount==0):
                    posi=j
                oneCount += 1
            elif(oneCount>=1):
                M[i, posi: j]=oneCount
                oneCount=0
        if(M[i,j]==1):
            M[i, posi: j+1]=oneCount
            oneCount=0
    return M

In [ ]:
Matrix=f(Matrix)

In [ ]:
if(np.max(Matrix)<=127):
    Matrix=Matrix.astype('int8')

In [ ]:
Matrix=Matrix.T

反復配列の検出

In [ ]:
for j in [NF1, NF2]:
    if(j!=0):   
        noise_free=j
        Matrix=np.where(Matrix<=j,0,Matrix)
    repeated_sequence=np.zeros(Hight)
    for i in range(Hight):
        repeated_sequence[i]=np.sum(Matrix[i])
    if(j==NF1):
        repeated_sequence_NF1=repeated_sequence
    if(j==NF2):
        repeated_sequence_NF2=repeated_sequence

del Matrix
gc.collect()

反復配列の領域を求める

In [ ]:
a=repeated_sequence_NF2.copy()

In [ ]:
b=np.where(a>0,1,a)

#ノイズ除去
c=np.zeros(len(b))
for i in range(L1, len(b)-L1):
    if(b[i]==1):
        left=( np.count_nonzero(b[i-L1: i]==1) )/L1
        right=( np.count_nonzero(b[i+1: i+L1+1]==1) )/L1
        if( left<=density and right<=density ):
            c[i]=-1
b=(b+c).copy()

#領域検出
c=np.zeros(len(b))
for i in range(L1_R, len(b)-L1_R):
    if(b[i]==0):
        if( (1 in b[i-L1_R: i]) and (1 in b[i+1: i+L1_R+1]) ):
            c[i]=1
b=(b+c).copy()

b[0]=0
b[-1]=0
b_diff=np.roll(b,-1)-b
startArray=np.where(b_diff==1)[0]
endArray=np.where(b_diff==-1)[0]

In [ ]:
plt.figure(figsize=(20,10))
plt.title(name)
plt.plot(a)
plt.plot(b*np.max(a))
plt.savefig(name)
plt.close()

領域に含まれる反復配列の個数を求める

In [ ]:
a=repeated_sequence_NF1.copy()

In [ ]:
b=np.zeros(len(a))    
for i in range(L2,len(a)):
    if(i+L2+1>len(a)):
        break
    else:
        b[i] = a[i-L2: i+L2+1].mean()
average_count=b.copy()

In [ ]:
a=average_count.copy()

In [ ]:
E_value=0
for i in range(NF2+1, 1000):
    E_value = E_value + (9/16)*( (1/4)**i )*(i**2)

In [ ]:
if(len(startArray)==0):
    with open('RSD.txt', mode='a') as f:
        f.write('\tnot exist1')
else:
    continueCounter=0
    for j in range(0,len(startArray)):
        startIndex=startArray[j].astype(np.int)
        endIndex=endArray[j].astype(np.int)
        
        if(startIndex<=L2):
            startIndex=L2
        if(endIndex>=len(seq)-L2-1):
            endIndex=len(seq)-L2-1        
        if( (endIndex-startIndex) < minLength ):
            continueCounter=continueCounter+1
            continue
        else:
            if( (endIndex-startIndex)<lengthOfRepeat/Ratio ):
                count_e=1
                First=1
                candidate=1
                Length=endIndex-startIndex
            else:
                count_e = np.round( (endIndex-startIndex) / lengthOfRepeat ).astype(np.int)
                #以下はフーリエ変換
                A = a[startIndex : endIndex]
                X=( abs(np.fft.fft(A))/len(A) )*2
                X[0]=0
                for k in range(N_fourier):
                    First = np.argmax(X[0: math.floor(len(A)/2)])
                    Length = len(A) / First
                    Length = math.floor(Length)
                    if( lengthOfRepeat/Ratio <= Length <= lengthOfRepeat*Ratio):
                        break
                    else:
                        X[First]=0
                candidate=k+1
                
                if(k==N_fourier-1):
                    if( (Length < lengthOfRepeat/Ratio) or (Length > lengthOfRepeat*Ratio) ):
                        First=0
                        candidate=0
                        Length=endIndex-startIndex

            determinant = np.log10( repeated_sequence_NF2[startIndex: endIndex].mean()/ (E_value*len(seq)) )
            if(j==continueCounter):        
                with open('RSD.txt', mode='a') as f:
                    f.write('\t'+str(NF2)+
                            '\t'+str(startIndex)+
                            '\t'+str(endIndex)+
                            '\t'+str(count_e)+
                            '\t'+str(First)+
                            '\t'+str(candidate)+
                            '\t'+str(Length)+
                            '\t'+str(determinant))                
            else:
                with open('RSD.txt', mode='a') as f:
                    f.write('\n'+name+
                            '\t'+str(len(seq))+
                            '\t'+str(NF2)+
                            '\t'+str(startIndex)+
                            '\t'+str(endIndex)+
                            '\t'+str(count_e)+
                            '\t'+str(First)+
                            '\t'+str(candidate)+
                            '\t'+str(Length)+
                            '\t'+str(determinant))
    if(continueCounter==len(startArray)):
        with open('RSD.txt', mode='a') as f:
            f.write('\tnot exist2')       